# A Director's impact on the bottom line

We primarily concerned with identifying any trends in the data regarding directors and top performing films. Key data points considered are film ratings, IMDB general film details and net revenue.

note: to create the movie.db locally, run the notebook SQLiteDatabase.ipynb

In [2]:
#necessary imports
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from glob import glob

#remove column truncation 
pd.set_option('display.max_columns', None)

## Correlations between films with the highest net revenue, ratings and directors

### What are the highest net revenue films?

In [3]:
conn = sqlite3.connect('movie.db') 
cur = conn.cursor()

cur.execute(""" 
SELECT 
  t.year as release_year, SUM(t.worldwide_gross-t.production_budget) as netRevenue_$,
  t.movie as film_title
FROM tn_movie_budgets t 
GROUP BY t.movie
ORDER BY netRevenue_$ DESC
LIMIT 5
;
""")

dft = pd.DataFrame(cur.fetchall())
dft.columns = (x[0] for x in cur.description)
#print(dft.shape)
#print(dft.info())

dft

,release_year,netRevenue_$,film_title
0,2009,2351345279,Avatar
1,1997,2008208395,Titanic
2,2018,1748134200,Avengers: Infinity War
3,2015,1747311220,Star Wars: Episode VII - The Force Awakens
4,2017,1687630838,Beauty and the Beast


The top 5 net revenue films are:

1. Avatar

2. Titanic

3. Avengers: Infinity War

4. Star Wars Ep. VII: The Force Awakens

5. Beauty and the Beast

#### Of the highest net revenue films, who are the directors?

In [5]:
conn = sqlite3.connect('movie.db') 
cur = conn.cursor()

cur.execute(""" 
SELECT c.tconst as movieID, nm.known_for_titles as knownFor_movieID, 
i.primary_title as movie_title, nm.primary_name as Director, 
nm.primary_profession, nm.birth_year
FROM imdb_title_crew c
LEFT JOIN imdb_title_basics i
ON c.tconst = i.tconst
LEFT JOIN tn_movie_budgets t 
ON i.primary_title = t.movie
LEFT JOIN imdb_name_basics nm
ON c.directors = nm.nconst OR c.director1 = nm.nconst OR 
c.director2 = nm.nconst
WHERE (i.primary_title == 'Beauty and the Beast') & (i.tconst == 'tt2771200') 
& (t.year like '%17%') OR (i.primary_title == 'Avatar') OR 
(i.primary_title == 'Titanic') OR (i.primary_title like '%Infinity War%') & 
(i.primary_title like '%Avengers%') OR (i.primary_title like '%Star War%') & 
(i.primary_title like '%Ep%') & (i.primary_title like '%Force Awakens%')
ORDER BY c.director1,c.director2,c.director3,c.director4
;
""")

dft = pd.DataFrame(cur.fetchall())
dft.columns = (x[0] for x in cur.description)
#print(dft.shape)
#print(dft.info())

dft

,movieID,knownFor_movieID,movie_title,Director,primary_profession,birth_year
0,tt1775309,"tt1594549,tt1775309,tt1602612",Avatar,Atsushi Wada,director,NaN
1,tt2771200,"tt0362269,tt0443489,tt0120684,tt0299658",Beauty and the Beast,Bill Condon,"director,writer,producer",1955.0
2,tt2488496,"tt0411008,tt0796366,tt1650062,tt2488496",Star Wars: Episode VII - The Force Awakens,J.J. Abrams,"producer,writer,music_department",1966.0
3,tt8852130,tt8852130,Titanic,Ravi Punj,"director,writer,producer",NaN
4,tt4154756,"tt1843866,tt4154756,tt3498820,tt4154796",Avengers: Infinity War,Anthony Russo,"producer,director,writer",1970.0
5,tt4154756,"tt4154756,tt4154796,tt1843866,tt3498820",Avengers: Infinity War,Joe Russo,"producer,director,writer",1971.0
6,tt2495766,"tt3381008,tt3171832,tt3495026,tt2495766",Titanic,Pete Meads,"actor,camera_department,miscellaneous",NaN
7,tt2495766,tt2495766,Titanic,Kevin Lincoln,"director,actor,writer",NaN


***Findings***: The directors are:
* Atsushi Wada, 'Avatar'
* Anthony Russo and Joe Russo, 'Avengers: Infinity War'
* Bill Condon, 'Beauty and the Beast'
* J.J. Abrams, 'Star Wars: Episode VII - The Force Awakens'
* Ravi Punj, Pete Meads and Kevin Lincoln, 'Titanic'


#### Are these directors well known for their work on these films?

***Findings***: Nearly all the directors are well known for their work on the top net revenue films. By reviewing whether the movieID is contained in the list of knownFor_movieID, we can see that 'Beauty and the Beast' is the outlier. Bill Condon is well known for other films but not for his work on this particular film.

### What are the top flims as rated by viewers?

In [10]:
conn = sqlite3.connect('movie.db') 
cur = conn.cursor()

cur.execute(""" 
SELECT title, vote_average as viewer_Ratings
FROM tmdb_movies
WHERE vote_count >= '10000'
ORDER BY vote_average DESC;""")

pd.set_option('display.max_columns', None)
dft = pd.DataFrame(cur.fetchall())
dft.columns = (x[0] for x in cur.description)
#print(dft.shape)
#print(dft.info())
dft.head()

,title,viewer_Ratings
0,Inception,8.3
1,Avengers: Infinity War,8.3
2,The Lion King,8.2
3,Interstellar,8.2
4,Shutter Island,8.1


***Findings***: We can see that the top 5 user-rated films from TheMovieDB.org are:
1. Inception

2. Avengers: Infinity Wars

3. The Lion King

4. Interstellar

5. Shutter Island

Of these top user-rated films, the only film that is also in the top 5 films for net revenue is 'Avengers: Infinity Wars'.

#### Of the top-rated films, who are the directors?

In [11]:
conn = sqlite3.connect('movie.db') 
cur = conn.cursor()

cur.execute(""" 
SELECT t.title as movie_title, t.vote_average as viewer_Ratings, nm.primary_name as Director, 
nm.primary_profession, nm.birth_year
FROM tmdb_movies t
LEFT JOIN imdb_title_basics b
ON t.title = b.primary_title
LEFT JOIN imdb_title_crew c
USING (tconst)
LEFT JOIN imdb_name_basics nm
ON c.directors = nm.nconst OR c.director1 = nm.nconst OR 
c.director2 = nm.nconst
WHERE t.vote_count >= '10000'
ORDER BY vote_average DESC
LIMIT 6;""")

pd.set_option('display.max_columns', None)
dft = pd.DataFrame(cur.fetchall())
dft.columns = (x[0] for x in cur.description)
#print(dft.shape)
#print(dft.info())
dft

,movie_title,viewer_Ratings,Director,primary_profession,birth_year
0,Inception,8.3,Christopher Nolan,"writer,producer,director",1970
1,Avengers: Infinity War,8.3,Anthony Russo,"producer,director,writer",1970
2,Avengers: Infinity War,8.3,Joe Russo,"producer,director,writer",1971
3,The Lion King,8.2,Jon Favreau,"producer,actor,miscellaneous",1966
4,Interstellar,8.2,Christopher Nolan,"writer,producer,director",1970
5,Shutter Island,8.1,Martin Scorsese,"producer,director,actor",1942


#### Are these directors well known for their work on these films?

In [8]:
conn = sqlite3.connect('movie.db') 
cur = conn.cursor()

cur.execute(""" 
SELECT c.tconst as movieID, nm.known_for_titles as knownFor_movieID, 
b.primary_title as movie_title, nm.primary_name as Director, 
nm.primary_profession, nm.birth_year
FROM imdb_title_crew c
LEFT JOIN imdb_title_basics b
ON c.tconst = b.tconst
LEFT JOIN tn_movie_budgets t 
ON b.primary_title = t.movie
LEFT JOIN imdb_name_basics nm
ON c.directors = nm.nconst OR c.director1 = nm.nconst OR 
c.director2 = nm.nconst
WHERE (b.primary_title == 'Inception') OR (b.primary_title == 'The Lion King') OR 
(b.primary_title == 'Interstellar') OR (b.primary_title like '%Infinity War%') 
& (b.primary_title like '%Avengers%') OR (b.primary_title == 'Shutter Island')
ORDER BY c.director1,c.director2,c.director3,c.director4
;
""")

dft = pd.DataFrame(cur.fetchall())
dft.columns = (x[0] for x in cur.description)
#print(dft.shape)
#print(dft.info())

dft

,movieID,knownFor_movieID,movie_title,Director,primary_profession,birth_year
0,tt1375666,"tt0482571,tt1375666,tt5013056,tt0154506",Inception,Christopher Nolan,"writer,producer,director",1970
1,tt0816692,"tt0482571,tt1375666,tt5013056,tt0154506",Interstellar,Christopher Nolan,"writer,producer,director",1970
2,tt6105098,"tt0117802,tt2883512,tt4154796,tt0371746",The Lion King,Jon Favreau,"producer,actor,miscellaneous",1966
3,tt1130884,"tt0490215,tt0075314,tt0099685,tt0070379",Shutter Island,Martin Scorsese,"producer,director,actor",1942
4,tt4154756,"tt1843866,tt4154756,tt3498820,tt4154796",Avengers: Infinity War,Anthony Russo,"producer,director,writer",1970
5,tt4154756,"tt4154756,tt4154796,tt1843866,tt3498820",Avengers: Infinity War,Joe Russo,"producer,director,writer",1971


***Findings***: Only two movie titles have directors who are are well known for their work on these top user-rated films:
1. Avengers: Infinity War

2. Inception

In [9]:
conn = sqlite3.connect('movie.db') 
cur = conn.cursor()

cur.execute(""" 
SELECT 
  t.year as release_year, SUM(t.worldwide_gross-t.production_budget) as netRevenue_$, 
  t.production_budget as productCost_$, t.movie as film_title
FROM tn_movie_budgets t 
WHERE (t.movie == 'Interstellar') OR (t.movie == 'Inception')
GROUP BY t.movie
ORDER BY netRevenue_$ DESC
LIMIT 5
;
""")

dft = pd.DataFrame(cur.fetchall())
dft.columns = (x[0] for x in cur.description)
#print(dft.shape)
#print(dft.info())

dft

,release_year,netRevenue_$,productCost_$,film_title
0,2010,675524642,160000000,Inception
1,2014,501379375,165000000,Interstellar


***Additional Findings***: Interestingly, the director for 'Inception', Christopher Nolan, also directed another top user-rated film 'Interstellar' but this film is not a top net revenue film. 'Interstellar' had nearly the same production cost as 'Inception' but trailed in net revenue by \\$17.4M.

## Data Visualizations